In [ ]:
from xfvcom import FvcomDataLoader, FvcomPlotConfig, FvcomPlotter
from xfvcom.helpers import FrameGenerator, create_gif
import pandas as pd

In [ ]:
def add_custom_plot(ax):
    ax.set_title("Title with Custom Plot")

In [ ]:
# Loading FVCOM output netcdf
base_path = "~/Github/TB-FVCOM/goto2023/output"
# List of netcdf files convenient to switch to another netcdf by specifying its index
ncfiles = ["TokyoBay18_r16_crossed_0001.nc"]
index_ncfile = 0
fvcom = FvcomDataLoader(base_path=base_path, ncfile=ncfiles[index_ncfile], time_tolerance=5)

# Set up FvcomPlotter
plotter = FvcomPlotter(fvcom.ds, FvcomPlotConfig(figsize=(6, 8)))

var_name = "salinity"
plot_kwargs={"verbose": False, "vmin": 10, "vmax": 20, "levels": [9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 15, 16, 17, 18, 19, 20]}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20}
plot_kwargs={}

da = fvcom.ds[var_name].isel(siglay=0)
start_date = da.time.isel(time=0).values
start_date = pd.to_datetime(start_date).strftime("%Y%m%d")
end_date = da.time.isel(time=-1).values
end_date = pd.to_datetime(end_date).strftime("%Y%m%d")

suffix = "_frame"
len_suffix = len(suffix)
base_name = f"{da.long_name}_{start_date}-{end_date}{suffix}"

# Generate movie frames using helper methods in helpers.py
output_dir = "frames"

frames = FrameGenerator.generate_frames(data_array=da, output_dir=output_dir, plotter=plotter, base_name=base_name,
    post_process_func=add_custom_plot, plot_kwargs=plot_kwargs)

# Create GIF animation
output_gif = f"{base_name[:-len_suffix]}.gif"
plotter.create_gif(frames, output_gif, fps=10)

## Example with customizing `plot_data`

In [ ]:
class CustomFrameGenerator(FrameGenerator):
    @staticmethod
    def plot_data(data_array, time, plotter, save_path, post_process_func=None, plot_kwargs=plot_kwargs):
        """
        Custom data plotting logic.
        """
        if post_process_func is not None:
            raise ValueError("post_process_func is not allowed to be defined as it must be defined in plot_data in class CustomFrameGenerator.")
        # Retreive DataArray
        da = data_array.isel(time=time)

        datetime = pd.Timestamp(da.time.item()).strftime('%Y-%m-%d %H:%M:%S')
        print(datetime)
        def add_custom_plot(ax):
            """
            Custom plot function to modify ax
            """
            ax.set_title(datetime)
            return ax
        
        plotter.plot_2d(da=da, save_path=save_path, post_process_func=add_custom_plot, **plot_kwargs)

    @staticmethod
    def generate_frame(args):
        """
        Reuse the parent class's generate_frame method.
        """
        # 親クラスのgenerate_frameを呼び出し
        return super(CustomFrameGenerator, CustomFrameGenerator).generate_frame(args)


In [ ]:
#from custom_helpers import CustomFrameGenerator  # カスタムクラス

# 必要なデータを準備
# Loading FVCOM output netcdf
base_path = "~/Github/TB-FVCOM/goto2023/output"
# List of netcdf files convenient to switch to another netcdf by specifying its index
ncfiles = ["TokyoBay18_r16_crossed_0001.nc"]
index_ncfile = 0
fvcom = FvcomDataLoader(base_path=base_path, ncfile=ncfiles[index_ncfile], time_tolerance=5)

# Set up FvcomPlotter
plotter = FvcomPlotter(fvcom.ds, FvcomPlotConfig(figsize=(6, 8)))

var_name = "salinity"
plot_kwargs={"verbose": False, "vmin": 10, "vmax": 20, "levels": [9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 15, 16, 17, 18, 19, 20]}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20}
#plot_kwargs={}

da = fvcom.ds[var_name].isel(siglay=0)
start_date = da.time.isel(time=0).values
start_date = pd.to_datetime(start_date).strftime("%Y%m%d")
end_date = da.time.isel(time=-1).values
end_date = pd.to_datetime(end_date).strftime("%Y%m%d")

suffix = "_frame"
len_suffix = len(suffix)
base_name = f"{da.long_name}_{start_date}-{end_date}{suffix}"

# Generate movie frames using helper methods in helpers.py
output_dir = "frames"

# カスタムフレームジェネレータを使ってフレームを生成
frames = CustomFrameGenerator.generate_frames(
    data_array=da, output_dir=output_dir, plotter=plotter, base_name=base_name,
    post_process_func=None, **plot_kwargs)

# Create GIF animation
output_gif = f"{base_name[:-len_suffix]}.gif"
create_gif(frames, output_gif, fps=10)

In [ ]:
da.time[0]